In [1]:
import pandas as pd

file_path = './data/driving.csv'
df = pd.read_csv(file_path)

df['Record_ID'] = (df['Time(s)'] == 1).cumsum()
df[['Class', 'PathOrder', 'Time(s)', 'Record_ID']]

,Class,PathOrder,Time(s),Record_ID
0,A,1,1,1
1,A,1,2,1
2,A,1,3,1
3,A,1,4,1
4,A,1,5,1
...,...,...,...,...
94375,D,2,2564,44
94376,D,2,2565,44
94377,D,2,2566,44
94378,D,2,2567,44


In [20]:
def sliding_window_by_record(df, record_col, time_col, window_size, step_size):
    record_windows = {}  # Record_ID별 윈도우 리스트를 저장할 딕셔너리

    for record_id, group in df.groupby(record_col):
        windows = []  # 각 Record_ID별 슬라이딩 윈도우를 저장할 리스트
        max_index = len(group) - window_size + 1  # 가능한 최대 인덱스
        
        for start in range(0, max_index, step_size):
            window = group.iloc[start:start + window_size].reset_index(drop=True)
            windows.append(window)
        
        # Record_ID별 윈도우 리스트를 딕셔너리에 저장
        record_windows[record_id] = windows

    return record_windows

In [21]:
window_size = 10
step_size = 1

record_windows = sliding_window_by_record(df, 'Record_ID', 'Time(s)', window_size, step_size)

class_windows = {}

for record_id, windows in record_windows.items():
    for window in windows:
        y_label = window['Class'].iloc[0]  # Class 열의 첫 번째 값을 y로 설정
        X_window = window.drop(columns=['Record_ID', 'PathOrder', 'Class', 'Time(s)'])  # X 값으로 사용할 열만 남김
        
        # y_label을 키로 하고, X_window 리스트에 추가
        if y_label not in class_windows:
            class_windows[y_label] = []  # y_label이 처음 등장하면 리스트 초기화
        class_windows[y_label].append(X_window)  # 모든 윈도우 추가

In [22]:
class_windows['A'][3]

,Fuel_consumption,Accelerator_Pedal_value,Throttle_position_signal,Short_Term_Fuel_Trim_Bank1,Intake_air_pressure,Filtered_Accelerator_Pedal_value,Absolute_throttle_position,Engine_soacking_time,Inhibition_of_engine_fuel_cut_off,Engine_in_fuel_cut_off,...,Converter_clutch,Gear_Selection,Vehicle_speed,Acceleration_speed_-_Longitudinal,Indication_of_brake_switch_ON/OFF,Master_cylinder_pressure,Calculated_road_gradient,Acceleration_speed_-_Lateral,Steering_wheel_speed,Steering_wheel_angle
0,204.8,0.0,4.7,0.0,38,0,13.3,3,0,0,...,0,0,0,0.1,1,0.9,0.0,-0.2,0,-3.6
1,217.6,0.0,5.7,0.0,40,0,13.7,3,0,0,...,0,0,0,0.1,1,0.9,0.0,-0.2,0,-3.5
2,243.2,0.0,5.7,0.0,41,0,13.7,3,0,0,...,0,0,0,0.0,2,36.7,0.0,-0.2,0,-3.4
3,217.6,0.0,5.7,0.0,42,0,13.7,3,0,0,...,0,7,0,0.0,2,61.8,0.0,-0.2,0,-3.6
4,294.4,0.0,6.6,0.0,52,0,14.5,3,0,0,...,0,7,0,0.0,2,38.9,0.0,-0.2,0,-1.6
5,332.8,0.0,7.1,0.0,60,0,14.5,3,0,0,...,0,7,0,0.0,2,12.5,0.0,-0.2,8,7.9
6,358.4,0.0,7.1,0.8,65,0,14.9,3,0,0,...,0,7,0,-0.1,2,1.8,0.0,-0.2,0,7.7
7,409.6,0.0,7.5,3.9,69,0,15.3,3,0,0,...,0,7,0,-0.2,2,1.4,0.0,-0.2,0,5.9
8,435.2,0.0,7.1,3.9,69,0,14.9,3,0,0,...,0,7,0,-0.1,2,2.4,0.0,-0.2,0,4.8
9,384.0,0.0,7.1,3.1,66,0,14.9,3,0,0,...,0,7,0,0.0,2,3.2,0.0,-0.2,8,-7.3
